In [5]:
import tensorflow as tf
import argparse
import os.path
from cifar10_models import *
from cifar10_input import *
from utils import *
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

#### SET THIS BEFORE RUNNING
NAME = "cifar10/cifar10_scm_001"
try:
    os.makedirs(CHECKPOINT.format(NAME))
except FileExistsError:
    pass

tf.reset_default_graph()
train_files, test_files = maybe_download_and_extract()
train_data = Cifar10Record(train_files, epochs=1)
test_data = Cifar10Record(test_files, epochs=1)

model = Cifar10ShallowConvolutionalModel(test_data.image, test_data.label)
model = Cifar10ShallowConvolutionalModel(train_data.image, train_data.label)

try:
    checkpoints = sorted(["{}/{}".format(ROOT.format(NAME),x[:-5]) for x in os.listdir(ROOT.format(NAME)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
except FileNotFoundError:
    checkpoints = []
model.saver.recover_last_checkpoints(checkpoints)
try: latest_checkpoint = model.saver.last_checkpoints[-2]
except: latest_checkpoint = None

In [6]:
acc = []
with tf.Session() as sess:
    model.saver.restore(sess, latest_checkpoint)
    while True:
        try:
            acc.append(model.accuracy.eval())
        except Exception e:
            print(e)
            break

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_001/model.ckpt-359000
<class 'Exception'>


In [4]:
print(len(acc),
    np.mean(acc))

100 0.6512


In [7]:
print(len(acc), np.mean(acc))

500 1.0


In [5]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

# List ALL tensors example output: v0/Adam (DT_FLOAT) [3,3,1,80]
print_tensors_in_checkpoint_file(file_name=latest_checkpoint, tensor_name='', all_tensors=True)

tensor_name:  cifar10_scm_1/layer_1_conv/bias
[-0.01462072  0.1017798   0.08205348  0.10501415  0.14708947  0.15814814
  0.01483207  0.12490834  0.0421839   0.01899061  0.06846154  0.1986398
  0.18543497 -0.02301033  0.20834765  0.14724943  0.17759813  0.2968531
  0.20604581  0.07048125  0.16239634  0.16688091  0.20778854  0.23207219
 -0.01957085  0.14875515  0.20536681  0.17245437  0.14744052  0.19602817
  0.11573488  0.23261514]
tensor_name:  cifar10_scm_1/layer_1_conv/weights
[[[[ -2.01392412e-01  -4.20100763e-02   2.55736168e-02 ...,
      5.68723343e-02   6.15832768e-02  -5.80123514e-02]
   [ -1.84509024e-01   2.61296481e-02  -8.77230018e-02 ...,
     -1.21440999e-01   4.86129001e-02   1.17014693e-02]
   [ -3.59960683e-02   1.54593721e-01   1.29467612e-02 ...,
      2.33200043e-01  -7.27953687e-02   2.01874048e-01]]

  [[  3.63187604e-02  -2.17923865e-01  -6.73186332e-02 ...,
     -2.67287493e-02  -1.57423884e-01  -3.02569807e-01]
   [  1.53998777e-01  -2.02590480e-01  -1.00499325